In [ ]:
import requests
import json
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
VERSION = '5.63'
API_ADDRESS = 'https://api.vk.com/method/'
ACCESS_TOKEN = '76e01e83604e3f9e6527ab13fb0320d4cefd4fa8424a93c3f6032c1f94096f9d09b7fd1bcd0954a9b54db'
METHOD = 'groups.getMembers'
METHOD_EX = 'execute?access_token=' + ACCESS_TOKEN + '&code='

In [ ]:
g_lst = pd.read_json('top100.json')
group_list = g_lst['id'].head(5).tolist()

In [ ]:
def get_members_count(id_of_group):
    params = {'group_id': id_of_group, 'access_token': ACCESS_TOKEN, 'v': VERSION, 'fields':'sex, bdate'}
    response = requests.get(API_ADDRESS + METHOD, params)
    r = response.json()['response']['count']
    return r

In [ ]:
print("Количество подписчиков в первых 5-ти группах:")
for i in group_list:
    print(get_members_count(i))

In [ ]:
def get_group_members(id_of_group, n=25000):
    offset_parm = 0
    for i in range(get_members_count(id_of_group) // n + 1):
        code = 'return ['
        for i in range(25):
            code = '%s%s' % (code, 'API.groups.getMembers({"group_id":%s, "fields":"sex, bdate", "offset":%s}),' % (id_of_group, offset_parm)) 
            offset_parm += 1000
        code = '%s%s' % (code, '];')
        response1 = requests.get(API_ADDRESS + METHOD_EX + code).json()['response']
        for i in response1:
            cleaned_members.extend(i['users'])
        print(offset_parm)
        print("Количество человек, выбранных для анализа:", len(cleaned_members))

In [ ]:
cleaned_members = []

При необходимости выбрать ВСЕХ подписчиков личности параметр **n** в функции **get_group_members** должен быть равен **25000**. Так как за один запрос выбирается 25000 пользователей.

In [ ]:
for i in group_list:
    get_group_members(i, n=2500000)
    offset_parm = 0

In [ ]:
members = pd.DataFrame(cleaned_members)

In [ ]:
del members['uid']
del members['first_name']
del members['last_name']
del members['deactivated']
members = members.replace(to_replace=[1, 2], value=['F', 'M'])
members['count'] = 1
members_gender = members[members.sex != 0]

In [ ]:
members_gender.groupby('sex').sum().plot.bar()

In [ ]:
members_age = members['bdate'].str[:].str.split('.', expand=True)

In [ ]:
members_age = members_age.rename(columns={0:'day', 1:'m', 2: 'year'})
members_age['age'] = 0
members_age = members_age[pd.notnull(members_age['year'])]

In [ ]:
members_age['age'] = 2017 - members_age['year'].astype('int')

In [ ]:
bins = [10, 15, 20, 29, 39, 79, 200]

In [ ]:
age_names = ['10-15', '16-20', '21-30', '30-40', '40-80', '80 to eternity']

In [ ]:
categories = pd.cut(members_age['age'], bins, labels=age_names)
members_age['categories'] = pd.cut(members_age['age'], bins, labels=age_names)

In [ ]:
pd.value_counts(members_age['categories']).sort_index().plot.bar()